In [1]:
# .env 정보 가져옴
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
naver_id = os.environ.get("NAVER_ID")
naver_pw = os.environ.get("NAVER_PW")

In [3]:
# 1 이메일 전송 함수 만들기

In [4]:
import os
import smtplib
# 이메일 메시지에 다양한 형식을 중첩하여 담기 위한 객체
from email.mime.multipart import MIMEMultipart

In [5]:
# 이메일 메시지를 이진 테이터로 바꿔주는 인코더
from email import encoders

In [6]:
# 텍스트 형식
from email.mime.text import MIMEText
# 이미지 형식
from email.mime.image import MIMEImage
# 오디오 형식
from email.mime.audio import MIMEAudio

In [7]:
# 위의 모든 객체를 생성할 수 있는 기본 객체
# MIMEBase(_maintype, _subtype)
# MIMEBase(<메인 타입>, <서브 타입>)
from email.mime.base import MIMEBase

In [8]:
def send_email(smtp_info, msg):
    with smtplib.SMTP(smtp_info["smtp_server"], smtp_info["smtp_port"]) as server:
        # TLS 보안 연결
        server.starttls()
        # 로그인
        server.login(smtp_info["smtp_user_id"], smtp_info["smtp_user_pw"])
        # 로그인 된 서버에 이메일 전송
        response = server.sendmail(msg["from"], msg["to"], msg.as_string())
        # 메시지를 보낼때는 .as_string() 메서드를 사용해서 문자열로 바꿔줌
        
        # 이메일을 성공적으로 보내면 결과는 {}
        if not response:
            print("이메일을 성공적으로 보냈습니다.")
        else:
            print(response)

In [9]:
def make_multimsg(msg_dict):
    multi = MIMEMultipart(_subtype="mixed")
    
    for key, value in msg_dict.items():
        # 각 타입에 적절한 MIMExxx() 함수를 호울하여 msg 객체를 생성
        if key == "text":
            with open(value["filename"], encoding="utf-8") as fp:
                msg = MIMEText(fp.read(), _subtype=value["subtype"])
        elif key == "image":
            with open(value["filename"], "rb") as fp:
                msg = MIMEImage(fp.read(), _subtype=value["subtype"])
        elif key == "audio":
            with open(value["filename"], "rb") as fp:
                msg = MIMEAudio(fp.read(), _subtype=value["subtype"])
        else:
            with open(value["filename"], "rb") as fp:
                msg = MIMEBase(value["maintype"], _subtype=value["subtype"])
                msg.set_payload(fp.read())
                encoders.encode_base64(msg)
        # 파일 이름을 첨부파일 제목으로 추가
        msg.add_header("Content-Disposition", "attachment", filename=os.path.basename(value["filename"]))
        # 첨부 파일 추가
        multi.attach(msg)
        
    return multi

In [10]:
# 2 텍스트로만 구성된 이메일 보내기

In [11]:
#from email.mime.text import MIMEText

In [12]:
smtp_info = dict({
    "smtp_server": "smtp.naver.com",
    "smtp_user_id": f"{naver_id}@naver.com",
    "smtp_user_pw": naver_pw,
    "smtp_port": 587,
})

In [13]:
# 메일 내용 작성
title = "기본 이메일입니다."
content = "메일 내용입니다."
sender = smtp_info["smtp_user_id"]  # 송신자(sender) 이메일 계정
receiver = "richdad02@naver.com"

In [14]:
# 메일 객체 생성: 메시지 내용에는 한글이 들어가기 때문에 한글을 지원하는 문자 체계인 UTF-8을 명시
msg = MIMEText(_text=content, _charset="utf-8")  # 이메일 내용

In [15]:
msg["Subject"] = title # 메일 제목
msg["From"] = sender   # 송신자
msg["to"] = receiver   # 수신자

In [16]:
send_email(smtp_info, msg)

이메일을 성공적으로 보냈습니다.


In [17]:
# 3 텍스트와 다양한 종류의 파일이 첨부된 이메일 보내기

In [18]:
msg_dict = {
    "text": {
        "maintype": "text",
        "subtype": "plain",
        "filename": "res/email_sending/test.txt",  # 텍스트 첨부 파일
    },
    "image": {
        "maintype": "image",
        "subtype": "jpg",
        "filename": "res/email_sending/test.jpg",  # 이미지 첨부 파일
    },
    "audio": {
        "maintype": "audio",
        "subtype": "mp3",
        "filename": "res/email_sending/test.mp3",  # 오디오 첨부 파일
    },
    "video": {
        "maintype": "video",
        "subtype": "mp4",
        "filename": "res/email_sending/test.mp4",  # 비디오 첨부 파일
    },
    "application": {
        "maintype": "application",
        "subtype": "octect-stream",
        "filename": "res/email_sending/test.pdf",  # 그 외 첨부 파일
    },
}

In [19]:
# 메일 내용 작성
title = "첨부파일이 있는 이메일 입니다."
content = "메일 내용입니다."
sender = smtp_info["smtp_user_id"]  # 송신자(sender) 이메일 계정
receiver = "richdad02@naver.com"

In [20]:
# 메일 객체 생성
msg = MIMEText(_text=content, _charset="utf-8")  # 이메일 내용

In [21]:
# 첨부파일 추가
multi = make_multimsg(msg_dict)
multi["Subject"] = title # 메일 제목
multi["From"] = sender   # 송신자
multi["to"] = receiver   # 수신자
multi.attach(msg)

In [22]:
# 첨부파일이 추가된 이메일 전송
send_email(smtp_info, multi)

이메일을 성공적으로 보냈습니다.
